## 데이터 수집

In [1]:
import matplotlib.pyplot as plt
import matplotlib as mpl # 한글 폰트 설정 (NanumGothic) 
mpl.rcParams['font.family'] = 'NanumGothic'
mpl.rcParams['axes.unicode_minus'] = False  # 마이너스 기호 깨짐 방지
import seaborn as sns
import pandas as pd
import numpy as np 
import scipy.stats as stats
from sklearn import datasets 
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

import lightgbm as lgb

In [2]:
data_df = pd.read_csv('../../datasets/titanic_disaster_train.csv')
data_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## 데이터 분석

In [3]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


- feature continue : Age, Fare
- feature category : Pclass, Sex, SibSp, Parch, Embarked
- label : Survived
- remove 
    + Cabin : 범주형인데 결측 너무 많아서 빼는게 나을 것으로 보임.
    + PassengerId : 유니크 함
    + Name : 유니크 함
    + Ticket : 유니크 함
- missing value : Age, Embarked

In [4]:
data_df['Survived'].value_counts()

Survived
0    549
1    342
Name: count, dtype: int64

## 데이터 전처리 

### 결측치 처리

##### Embarked
- 최대한 다른 변수에 영향을 안주는 범위에서 평균 제일 많은 값 채택 
- 새로 채우는 값이 다른 조건을 찾는 것에 영향 제일 안주도록 최대한 비슷한 패턴에서 평균값 취하기
- 최대한 생존했으며, 동승자 없고, ticket 번호 유사하고, 객실을 부여 받은 사람들
- 모두 S 이므로 S로 채움 

In [5]:
# 우선, Embarked
nan_embarked_records = data_df[data_df['Embarked'].isna()]
nan_embarked_records

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
61,62,1,1,"Icard, Miss. Amelie",female,38.0,0,0,113572,80.0,B28,NaN
829,830,1,1,"Stone, Mrs. George Nelson (Martha Evelyn)",female,62.0,0,0,113572,80.0,B28,NaN


In [6]:
data_df[data_df['Name'].str.contains('Stone', na=False)]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
319,320,1,1,"Spedden, Mrs. Frederic Oakley (Margaretta Corn...",female,40.0,1,1,16966,134.5,E34,C
829,830,1,1,"Stone, Mrs. George Nelson (Martha Evelyn)",female,62.0,0,0,113572,80.0,B28,NaN


In [7]:
data_df[data_df['Cabin'].notna()].head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
10,11,1,3,"Sandstrom, Miss. Marguerite Rut",female,4.0,1,1,PP 9549,16.7000,G6,S
11,12,1,1,"Bonnell, Miss. Elizabeth",female,58.0,0,0,113783,26.5500,C103,S


In [8]:
condition_ticket = "Ticket.str.match('^113\\d{3}$')" # 113572
data_df.query(f"{condition_ticket}")["Ticket"].value_counts()


Ticket
113781    4
113760    4
113803    2
113789    2
113572    2
113806    2
113776    2
113505    2
113798    2
113787    1
113050    1
113807    1
113796    1
113800    1
113804    1
113051    1
113028    1
113773    1
113501    1
113792    1
113503    1
113786    1
113794    1
113783    1
113510    1
113784    1
113043    1
113056    1
113514    1
113767    1
113059    1
113509    1
113788    1
113055    1
Name: count, dtype: int64

In [9]:
# 각 조건들을 변수로 정의
condition_pclass = "Pclass == 1"
condition_fare = "Fare >= 79.0 and Fare <= 82.0"
condition_SibSp = "SibSp == 0"
condition_Parch = "Parch == 0"
condition_Survived = "Survived == 1"
condition_ticket = "Ticket.str.match('^113\\d{3}$')" # 113572

# 모든 조건을 결합하여 쿼리 실행
# data_df[data_df['Cabin'].notna()].query(f"{condition_pclass} and {condition_fare} and {condition_SibSp} and {condition_cabin}")
data_df[data_df['Cabin'].notna()].query(f"{condition_Survived} and {condition_ticket}")
data_df[data_df['Cabin'].notna()].query(f"{condition_Survived} and {condition_ticket} and {condition_SibSp} and {condition_Parch}")


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
11,12,1,1,"Bonnell, Miss. Elizabeth",female,58.0,0,0,113783,26.55,C103,S
23,24,1,1,"Sloper, Mr. William Thompson",male,28.0,0,0,113788,35.50,A6,S
61,62,1,1,"Icard, Miss. Amelie",female,38.0,0,0,113572,80.00,B28,NaN
449,450,1,1,"Peuchen, Major. Arthur Godfrey",male,52.0,0,0,113786,30.50,C104,S
829,830,1,1,"Stone, Mrs. George Nelson (Martha Evelyn)",female,62.0,0,0,113572,80.00,B28,NaN
857,858,1,1,"Daly, Mr. Peter Denis",male,51.0,0,0,113055,26.55,E17,S


In [10]:
data_df['Embarked'] = data_df['Embarked'].fillna('S')
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     891 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [11]:
# age 는 ML 쓸거니까 다른거 먼저 처리하고 채우자

### 이상치 처리

### 범주형 처리

In [12]:
data_df = data_df.drop(columns=['Cabin', 'PassengerId', 'Ticket', 'Name'])
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    object 
 3   Age       714 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Fare      891 non-null    float64
 7   Embarked  891 non-null    object 
dtypes: float64(2), int64(4), object(2)
memory usage: 55.8+ KB


In [13]:
# 일단 제외 하고 가자 
# Survived 는 어차피 labelencode 해야해
# Age는 missing 이후에 처리
final_label_name = 'Survived'
missing_label_name = 'Age'

In [14]:
labelencoder = LabelEncoder()
labelencoder.fit(data_df[final_label_name])
encoded_label = labelencoder.transform(data_df[final_label_name])

In [ ]:
# encoded_label 이거 dataframe 만들기 
# pclass_df = pd.DataFrame(data=encoded_pclass, columns=pclass_name_list, index=data_df.index) 

In [15]:
missing_label = data_df[missing_label_name]

In [16]:
data_df = data_df.drop(columns=[final_label_name, missing_label_name])
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    891 non-null    int64  
 1   Sex       891 non-null    object 
 2   SibSp     891 non-null    int64  
 3   Parch     891 non-null    int64  
 4   Fare      891 non-null    float64
 5   Embarked  891 non-null    object 
dtypes: float64(1), int64(3), object(2)
memory usage: 41.9+ KB


In [17]:
def split_categorical_numerical(df, unique_threshold=0.05):
    """
    데이터프레임을 범주형과 수치형으로 분리
    
    Parameters:
    - df: 입력 데이터프레임
    - unique_threshold: unique 값 비율 임계값 (기본값 0.05 = 5%)
    
    Returns:
    - categorical_df: 범주형 컬럼만 있는 데이터프레임
    - numerical_df: 수치형 컬럼만 있는 데이터프레임
    """
    
    categorical_cols = []
    numerical_cols = []
    
    n_rows = len(df)
    
    for col in df.columns:
        # unique 값의 비율 계산
        unique_ratio = len(df[col].unique()) / n_rows
        
        # object 타입이거나, unique 값 비율이 임계값보다 작으면 범주형으로 분류
        # df[col].dtype == 'object' or
        if unique_ratio < unique_threshold:
            categorical_cols.append(col)
        else:
            numerical_cols.append(col)
            
    return df[categorical_cols], df[numerical_cols]

In [18]:
cat_df, num_df = split_categorical_numerical(data_df)
cat_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Pclass    891 non-null    int64 
 1   Sex       891 non-null    object
 2   SibSp     891 non-null    int64 
 3   Parch     891 non-null    int64 
 4   Embarked  891 non-null    object
dtypes: int64(3), object(2)
memory usage: 34.9+ KB


In [19]:
num_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Fare    891 non-null    float64
dtypes: float64(1)
memory usage: 7.1 KB


In [20]:
def run_onehotencode(data_df, model=OneHotEncoder()):

    columns = data_df.columns
    for col in columns :
        model.fit(X=data_df[[col]])
        encoded_pclass = model.transform(X=data_df[[col]]).toarray()
        pclass_name_list = model.get_feature_names_out(input_features=[col])
        pclass_df = pd.DataFrame(data=encoded_pclass, columns=pclass_name_list, index=data_df.index)
        # 원본 자르는거 추가 

        data_df = pd.concat([data_df.drop(columns=[col]), pclass_df], axis=1)

    return data_df

In [21]:
onehot_df = run_onehotencode(cat_df)
missing_df = pd.concat([onehot_df, num_df], axis=1)
missing_df.head()

,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,SibSp_0,SibSp_1,SibSp_2,SibSp_3,SibSp_4,...,Parch_1,Parch_2,Parch_3,Parch_4,Parch_5,Parch_6,Embarked_C,Embarked_Q,Embarked_S,Fare
0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,7.2500
1,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,71.2833
2,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,7.9250
3,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,53.1000
4,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,8.0500


In [22]:
missing_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 23 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Pclass_1    891 non-null    float64
 1   Pclass_2    891 non-null    float64
 2   Pclass_3    891 non-null    float64
 3   Sex_female  891 non-null    float64
 4   Sex_male    891 non-null    float64
 5   SibSp_0     891 non-null    float64
 6   SibSp_1     891 non-null    float64
 7   SibSp_2     891 non-null    float64
 8   SibSp_3     891 non-null    float64
 9   SibSp_4     891 non-null    float64
 10  SibSp_5     891 non-null    float64
 11  SibSp_8     891 non-null    float64
 12  Parch_0     891 non-null    float64
 13  Parch_1     891 non-null    float64
 14  Parch_2     891 non-null    float64
 15  Parch_3     891 non-null    float64
 16  Parch_4     891 non-null    float64
 17  Parch_5     891 non-null    float64
 18  Parch_6     891 non-null    float64
 19  Embarked_C  891 non-null    f

### 스케일링


## 데이터 분할

## 모델 학습 

## 모델 평가 

## 모델 배포